### Basic XGBoost Multiclassification Implementation on subset of features
#### https://github.com/dmlc/xgboost/blob/master/demo/multiclass_classification/train.py

In [35]:
import os
from collections import Counter
try:
    import xml.etree.cElementTree as ET
except ImportError:
    import xml.etree.ElementTree as ET
import numpy as np
from scipy import sparse

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn.metrics as metrics
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.cross_validation import train_test_split
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt
%matplotlib inline

In [36]:
# these are the fifteen malware classes we're looking for
malware_classes = ["Agent", "AutoRun", "FraudLoad", "FraudPack", "Hupigon", "Krap",
           "Lipler", "Magania", "None", "Poison", "Swizzor", "Tdss",
           "VB", "Virut", "Zbot"]

# a function for writing predictions in the required format
def write_predictions(predictions, ids, outfile):
    """
    assumes len(predictions) == len(ids), and that predictions[i] is the
    index of the predicted class with the malware_classes list above for 
    the executable corresponding to ids[i].
    outfile will be overwritten
    """
    with open(outfile,"w+") as f:
        # write header
        f.write("Id,Prediction\n")
        for i, history_id in enumerate(ids):
            f.write("%s,%d\n" % (history_id, predictions[i]))

In [37]:
import pandas as pd

ids_classes = []
trees = []
for fname in os.listdir('train'):
    if fname == '.DS_Store':
        continue
    id_str, clazz = fname.split('.')[:2]
    ids_classes.append((id_str, clazz))
    tree = ET.parse(os.path.join('train', fname))
    trees.append(tree)

train_df = pd.DataFrame.from_records(ids_classes, columns=['id','class']) 

In [38]:
train_df.head()

,id,class
0,00269ea50001a6c699d0222032d45b74b2e7e8be9,None
1,00278ec420236020d6121dffe0cc20034422e7228,Lipler
2,002d5615d19c851934dc481c607b6a74a6e9e536e,VB
3,006be5Dc265600c19728c9747fb4c7bc9e8d6f106,None
4,0089453df77890cae95ce7d9130a4ef85eaea36e8,Swizzor


In [39]:
y = train_df['class'].values

In [40]:
def to_2class(classes):
    return ['None' if label == 'None' else 'Mal' for label in classes] 

In [41]:
not_calls = ['processes','all_section','thread','process']
docs = []
for tree in trees:
    calls = []
    for ele in tree.iter():
        if ele.tag not in not_calls:
            calls.append(ele.tag)   
    docs.append(calls)

In [72]:
vectorizer = TfidfVectorizer(ngram_range=(1,5))
tfidf = vectorizer.fit_transform([' '.join(doc) for doc in docs])

In [73]:
X = pd.DataFrame(tfidf.toarray(), columns=vectorizer.get_feature_names())

In [ ]:
malware_classes = ["Agent", "AutoRun", "FraudLoad", "FraudPack", "Hupigon", "Krap",
           "Lipler", "Magania", "None", "Poison", "Swizzor", "Tdss",
           "VB", "Virut", "Zbot"]

In [98]:
y_list = []
for clazz in y:
    i=0
    for el in malware_classes:
        if clazz == el:
            y_list.append(i)
        i+=1
        
numerical_y = np.array(y_list)

In [137]:
import numpy as np
import xgboost as xgb

train_X, test_X, train_Y, test_Y = train_test_split(X, numerical_y)

xg_train = xgb.DMatrix( train_X, label=train_Y)
xg_test = xgb.DMatrix(test_X, label=test_Y)
# setup parameters for xgboost
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['eta'] = 0.1
param['max_depth'] = 6
param['silent'] = 1
#param['nthread'] = 4 #If not defined, it is set to maximum
param['num_class'] = 15
param["booster"] = "gbtree"
#param["lambda"] = 1 #default: 1
#param["alpha"] = 0 #default:0
#param["gamma"] = 0 # the larger the more conservative the model is

num_round = 5 # Given was 5 (Train error decreases with increasing rounds, maybe keep it small to avoid overfitting)
ntree=200 # Given was 6

watchlist = [ (xg_train,'train'), (xg_test, 'test') ]

#Tree
bst = xgb.train(param, xg_train, num_round, watchlist );
# get prediction
pred = bst.predict( xg_test , ntree_limit=ntree);

print ('predicting, classification error (Tree)=%f' % (sum( int(pred[i]) != test_Y[i] for i in range(len(test_Y))) / float(len(test_Y)) ))


#General Linear 
param["booster"] = "gblinear"
bst_linear = xgb.train(param, xg_train, num_round, watchlist );
# get prediction
pred_linear = bst_linear.predict( xg_test );

print ('predicting, classification error (Generalized Linear)=%f' % (sum( int(pred_linear[i]) != test_Y[i] for i in range(len(test_Y))) / float(len(test_Y)) ))




[0]	train-merror:0.072602	test-merror:0.116580
[1]	train-merror:0.060501	test-merror:0.110104
[2]	train-merror:0.050130	test-merror:0.102332
[3]	train-merror:0.045376	test-merror:0.090674
[4]	train-merror:0.039758	test-merror:0.091969
[0]	train-merror:0.027226	test-merror:0.101036
[1]	train-merror:0.058341	test-merror:0.147668
[2]	train-merror:0.143042	test-merror:0.196891
[3]	train-merror:0.732498	test-merror:0.787565
[4]	train-merror:0.369490	test-merror:0.362694


predicting, classification error (Tree)=0.091969
predicting, classification error (Generalized Linear)=0.362694
